In [1]:
from trino.dbapi import connect

conn = connect(
    host="trino.local.reinthal.cc",
    port=443,
    user="trino",
    catalog="lakehouse",
    schema="staging",
    http_scheme="https"
)
cur = conn.cursor()
cur.execute("SELECT * FROM lakehouse.staging.cit0day_password_files limit 5")
rows = cur.fetchall()

In [2]:
import polars as pl
pl.DataFrame(rows)

column_0,column_1,column_2,column_3,column_4
str,str,str,str,str
"""001.mustafa@gmail.com""","""007rahulpoonia@gmail.com""","""06neeru@gmail.com""","""1234@kiyara.com""","""143Rajdeep@Gmail.com"""
"""IPEM_5NE7UF""","""IPEM_DMPRTQ""","""IPEM_GSQSTT""","""CK11514""","""IPEM_2WIAKN"""
"""raw""","""raw""","""raw""","""raw""","""raw"""
"""extracted/Cit0day [_special_fo…","""extracted/Cit0day [_special_fo…","""extracted/Cit0day [_special_fo…","""extracted/Cit0day [_special_fo…","""extracted/Cit0day [_special_fo…"


In [3]:
from dagster import EnvVar
name = "default"
warehouse = EnvVar("NESSIE_WAREHOUSE").get_value()
branch = EnvVar("NESSIE_BRANCH").get_value()
uri = EnvVar("NESSIE_URI").get_value()
py_io_impl = "pyiceberg.io.pyarrow.PyArrowFileIO"
s3_endpoint = EnvVar("DESTINATION__FILESYSTEM__CREDENTIALS__AWS_S3_ENDPOINT").get_value()
s3_access_key_id = EnvVar(
    "DESTINATION__FILESYSTEM__CREDENTIALS__AWS_ACCESS_KEY_ID"
).get_value()
s3_secret_access_key = EnvVar(
    "DESTINATION__FILESYSTEM__CREDENTIALS__AWS_SECRET_ACCESS_KEY"
).get_value()
catalog_type = "rest"

In [6]:
print(branch)

feat-update-schema


In [7]:
from pyiceberg.catalog import Catalog, load_catalog

In [8]:
catalog = load_catalog(
            **{
                "warehouse": warehouse,
                "uri": f"{uri}/{branch}",
                "py-io-impl": py_io_impl,
                "s3.endpoint": s3_endpoint,
                "s3.access-key-id": s3_access_key_id,
                "s3.secret-access-key": s3_secret_access_key,
                "type": catalog_type,
            },
        )

In [9]:
catalog.list_tables("staging")

[('staging', 'cit0day_password_files')]

In [10]:
tbl = catalog.load_table("staging.cit0day_password_files")

In [11]:
scan = tbl.scan(limit=100)

In [ ]:
df = scan.to_arrow()

/nix/store/xnrpi91vnwq4vkz74lg7narjqv17i97l-python3-3.12.4-env/lib/python3.12/site-packages/pyiceberg/avro/decoder.py:185: UserWarning: Falling back to pure Python Avro decoder, missing Cython implementation
  warnings.warn("Falling back to pure Python Avro decoder, missing Cython implementation")


In [ ]:
df

In [52]:
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import DayTransform

In [53]:
from pyiceberg import transforms

In [54]:
??transforms

Type:        module
String form: <module 'pyiceberg.transforms' from '/nix/store/za65ln3gb4fc7sz7jyc3vwwjsa0hkrws-python3-3.12.4-env/lib/python3.12/site-packages/pyiceberg/transforms.py'>
File:        /nix/store/za65ln3gb4fc7sz7jyc3vwwjsa0hkrws-python3-3.12.4-env/lib/python3.12/site-packages/pyiceberg/transforms.py
Source:     
# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements.  See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership.  The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License.  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express o